In [4]:

import pickle 
with open('..\data\h_bar_annotation.pkl', 'rb') as handle:
    h = pickle.load(handle)

with open('..\data\s_bar_annotation.pkl', 'rb') as handle:
    s = pickle.load(handle)

print(len(h), len(s))

90 3


In [10]:
data = []
for i in h : data.append(i)
for i in s : data.append(i)
len(data)


93

In [13]:
import numpy as np

def find_isHbar(bboxes):

    bar_boxes = []
    isHbar = False

    for bb in bboxes:
        x1, y1, x2, y2 = bb[1]
        bar_boxes.append([float(x1), float(y1), float(x2), float(y2)])

    if len(bar_boxes) == 0:
        isHbar = False
        return isHbar

    x1_sorted = sorted(bar_boxes, key=lambda x:x[0])
    y2_sorted = sorted(bar_boxes, key=lambda x:x[3])

    x1_dist = []
    y2_dist = []

    for i in range(1, len(x1_sorted)):
        d = x1_sorted[i][0] - x1_sorted[i-1][0]
        x1_dist.append(d)

    for i in range(1, len(y2_sorted)):
        d = y2_sorted[i][3] - y2_sorted[i-1][3]
        y2_dist.append(d)

    if np.mean(x1_dist) >= np.mean(y2_dist):
        isHbar = False
    elif np.mean(x1_dist) < np.mean(y2_dist):
        isHbar = True

    return isHbar


In [ ]:
def find_legend_orientation(legend_preview_data):
    """
    Helper method to find the orientation of the legends.
    Inputs:
    legend_preview_data : (list of dictionary) A dictionary which has the bbox of the preview.
    Outputs:
    orientation: (string): Can be horizontal or vertical depending on how the legend-preview bboxes are oriented.
    """
    if len(legend_preview_data) > 1:
        center_x = []
        center_y = []
        for preview_bbox in legend_preview_data:
            x,y = find_center(preview_bbox['bbox'])
            center_x.append(x)
            center_y.append(y)

        if abs(center_x[1] - center_x[0]) > abs(center_y[1] - center_y[0]):
            orientation = 'horizontal'
        else:
            orientation = 'vertical'
    else:
        orientation = 'unknown'
    return orientation

def legend_preview_association(legend_preview_data, legend_label_data, orientation):
    """
    Helper method, used to associate the legend preview with the corresponding legend label.
    Inputs:
    legend_preview_data (list of dictionary): A dictionary which has the bbox of the preview and the associated color of the preview.
    legend_label_data (list of dictionary): A dictionary which has the bbox of the legend label and the associated label.
    orientation (string) : orientation of the legends (vertical or horizontal)
    Output:
    legend_preview_data (list of dictionary): A dictionary which has the bbox of the preview and the associated color of the preview and value as the associated legend label.
    Pseudocode:

    for all preview bboxes :
       for all legend-label bboxes :
          if preview_bbox_center_X < legend_label_bbox_center_X :
                find distance between the (xmax, ymax) of preview_bbox and (xmin,ymax) of label_bbox
       associate that legend-label to the preview bbox whose distance is minimum

    """
    if orientation == "vertical":
        # sort preview bboxes according to xmin
        legend_preview_data = sorted(legend_preview_data, key=lambda k: k['bbox'][1])
        # sort legend-label bboxes according to xmin and corresponding legend-labels also
        legend_label_data = sorted(legend_label_data, key=lambda k: k['bbox'][1])
    else:
        # sort preview bboxes according to xmin
        legend_preview_data = sorted(legend_preview_data, key=lambda k: k['bbox'][0])
        # sort legend-label bboxes according to xmin and corresponding legend-labels also
        legend_label_data = sorted(legend_label_data, key=lambda k: k['bbox'][0])

    # find the center of preview bboxes
    preview_bboxes_center = []
    for bbox in legend_preview_data:
        center_x, center_y = find_center(bbox['bbox'])
        preview_bboxes_center.append((center_x, center_y))

    # find the center of legend-label bboxes
    legend_label_bboxes_center = []
    for bbox in legend_label_data:
        center_x, center_y = find_center(bbox['bbox'])
        legend_label_bboxes_center.append((center_x, center_y))

    for p_idx, preview_bbox in enumerate(legend_preview_data):

        preview_xmax = preview_bbox['bbox'][2]
        preview_ymax = preview_bbox['bbox'][3]

        min_distance = 1000000
        min_lbl_idx = -1

        for lbl_idx, label_bbox in enumerate(legend_label_data):
            # compare with only those legends which are ahead of the preview
            if preview_bboxes_center[p_idx][0] < legend_label_bboxes_center[lbl_idx][0]:

                label_xmin = label_bbox['bbox'][0]
                label_ymax = label_bbox['bbox'][3]

                distance = ((preview_xmax - label_xmin)**2 + (preview_ymax - label_ymax)**2)**(0.5)

                if distance < min_distance:
                    min_distance = distance
                    min_lbl_idx = lbl_idx

        preview_bbox['associated_label'] = legend_label_data[min_lbl_idx]['ocr_text']

    return legend_preview_data

def associate_legend_preview(image_data):
    legend_preview_data = [dd for dd in image_data if dd["pred_class"] == "preview"]
    if len(legend_preview_data) > 1:
        legend_label_data = [dd for dd in image_data if dd["pred_class"] == "legend_label"]
        legend_orientation = find_legend_orientation(legend_preview_data)
        lpa = legend_preview_association(legend_preview_data, legend_label_data, legend_orientation)
        image_data = list_subtraction(image_data, legend_preview_data)
        image_data = image_data + lpa

    return image_data, legend_orientation

def associate_bar_legend(image_data, isHbar):

    preview_data = [dd for dd in image_data if dd["pred_class"] == "preview"]

    visual_data = [dd for dd in image_data if dd["pred_class"] in ["bar", "dot_line"]]

    image_data = list_subtraction(image_data, visual_data)

    # Grouping the visual elements based on tick labels
    visual_groups = form_groups(visual_data, isHbar)

    _mapping = {}
    updated_visual_data = []

    # Create a map from legend-label to corresponding preview-clor
    for i in range(len(preview_data)):
        c = preview_data[i]["color"]
        lbl = preview_data[i]["associated_label"]
        _mapping[lbl] = c

    # NITESH: for each group of visual elements, find the associated color
    for group in visual_groups:
        _group = match_colors(group, _mapping)
        for item in _group:
            updated_visual_data.append(item)
    #
    # for i in range(len(visual_data)):
    #     tmp_lbls = [lbl for lbl,c in mapping.items() if colorDistance(c, visual_data[i]["color"]) <= 20]
    #
    #     if len(tmp_lbls) > 0:
    #         visual_data[i]["associated_label"] = tmp_lbls[0]
    #     else:
    #         visual_data[i]["associated_label"] = "Unknown"

    image_data = image_data + updated_visual_data

    return image_data

from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
import copy
import random

def _colorDistance(c1, c2):
    '''
    Calculates the Euclidian distance between two colors
    '''
    x1, y1, z1 = c1
    x2, y2, z2 = c2
    d = ((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)**0.5
    return d

def colorDistance(c1, c2, method=""):
    if method == "euclidian":
        return _colorDistance(c1, c2)
    elif method == "delta_e":
        '''
        Computes delta_e_cie2000 distance between two colors
        '''
        color1_rgb = sRGBColor(c1[0], c1[1], c1[2]);
        color2_rgb = sRGBColor(c2[0], c2[1], c2[2]);

        # Convert from RGB to Lab Color Space
        color1_lab = convert_color(color1_rgb, LabColor);

        # Convert from RGB to Lab Color Space
        color2_lab = convert_color(color2_rgb, LabColor);

        # Find the color difference
        delta_e = delta_e_cie2000(color1_lab, color2_lab);

        return delta_e


def random_assignments(group1, group2):
    '''
    Inputs:
    group1 (list of dictionary) : visual elements to whom legend-label is not assigned
    group2 (dictionary) : a mapping which has legend-label as key and corresponding preview color as value
    '''
    for g in group1:
        if "associated_label" in g.keys():
            continue

        try:
            k = random.choice(group2.keys())
            del group2[k]
        except:
            k = "legend-label"
        g["associated_label"] = k

    return group1

def match_colors(group, _mapping):

    unassigned_visual_elements = []

    mapping = copy.deepcopy(_mapping)

    for dd in group:
        visual_color = dd["color"]

        if visual_color == [255, 255, 255]:
            unassigned_visual_elements.append(dd)
            continue

        # tmp_lbls = [lbl for lbl,c in mapping.items() if colorDistance(c, visual_color) <= 20]
        #
        # if len(tmp_lbls) > 0:
        #     dd["associated_label"] = tmp_lbls[0]
        #     del mapping[tmp_lbls[0]]
        # else:
        #     unassigned_visual_elements.append(dd)

        # change the way you associate the color to the visual items
        tmp_lbls = [lbl for lbl,c in mapping.items() if colorDistance(c, visual_color, method="euclidian") <= 20]
        distance_with_preview = [colorDistance(c, visual_color, method="euclidian") for lbl,c in mapping.items() if colorDistance(c, visual_color, method="euclidian") <= 20]

        if len(tmp_lbls) > 0:
            min_index_ = np.argmin(distance_with_preview)
            dd["associated_label"] = tmp_lbls[min_index_]
            del mapping[tmp_lbls[min_index_]]
        else:
            unassigned_visual_elements.append(dd)
    if len(unassigned_visual_elements):
        group = random_assignments(group, mapping)

    return group


In [18]:
def align_features(features, is_y = False, x_thres = 0.01, crlf_thres = 0.04):
    if is_y:
        features = sorted(features, key=lambda x: (np.round(x[2][1], 2), np.round(x[2][0], 2)))
    else:
        features = sorted(features, key=lambda x: (np.round(x[2][0], 2), np.round(x[2][1], 2)))
    temp_t, temp_c, temp_b, new_tcb = [], [], [], []
    cur_font_size, cur_x_mid, cur_y_max = 0, 0, 0
    for t, c, b in features:
        x_min, y_min, x_max, y_max = b
        x_mid = (x_min + x_max) / 2
        if len(temp_t) == 0:
            temp_t.append(t)
            temp_c.append(c)
            temp_b.append(b)
            cur_font_size = y_max - y_min
            cur_x_mid = x_mid
            cur_y_max = y_max
            continue
        if np.abs(x_mid - cur_x_mid) < x_thres and np.abs(y_min - cur_y_max) < cur_font_size/8:
            pass
        else:
            new_c = temp_c[0]
            new_b = np.array([min([x[0] for x in temp_b]), min([x[1] for x in temp_b]), max([x[2] for x in temp_b]), max([x[3] for x in temp_b])])
            y_orders = [y1 for x1, y1, x2, y2 in temp_b]
            sorted_idxes = np.argsort(y_orders)[::-1]
            new_t = ' '.join([temp_t[i] for i in sorted_idxes])
            new_tcb.append((new_t, new_c, new_b))
            temp_t, temp_c, temp_b = [], [], []
        temp_t.append(t)
        temp_c.append(c)
        temp_b.append(b)
        cur_font_size = y_max - y_min
        cur_x_mid = x_mid
        cur_y_max = y_max
    if len(temp_t) > 0:
        new_c =  temp_c[0]
        new_b = np.array([min([x[0] for x in temp_b]), min([x[1] for x in temp_b]), max([x[2] for x in temp_b]), max([x[3] for x in temp_b])])    
        y_orders = [y1 for x1, y1, x2, y2 in temp_b]
        sorted_idxes = np.argsort(y_orders)[::-1]
        new_t = ' '.join([temp_t[i] for i in sorted_idxes])
        new_tcb.append((new_t, new_c, new_b))

    return new_tcb

In [20]:

#rule based VED
def pred_bbox(bboxes):
    print(align_features(bboxes))


    

[('legend', None, array([1, 2, 3, 4])),
 ('legend', None, array([3, 6, 3, 6])),
 ('legend', None, array([5, 6, 7, 8]))]

In [17]:
import cv2
import xmltodict
import json 
import pickle 
import pandas as pd
import os 
from PIL import Image
from tqdm import tqdm 

text_label = ['title', 'y_label', 'x_label', 'x_tick', 'y_tick', 't_legend', 'value']
png_dir = "C:\\Users\\Dial\\Dropbox\\DIAL Lab\\C. etc\\Project\\2022 인공지능 정책 지원 AI 챌린지_1단계\\데이터셋\\221201 데이터 레이블링\\human"
for index, plot in enumerate(tqdm(data, ncols=60)):
    question, answer, image, bboxes_temp = plot['question'], plot['answer'], plot['image'], plot['bboxes'] 
    ##ASSUME WE DON'T KNOW VED LABELS
    bboxes=[]
    #ONLY USING TEXT ELEMENTS
    for bb in bboxes_temp:
        if bb[0] not in text_label: continue
        bboxes.append(bb)
    isHbar = find_isHbar(bboxes)

    image = Image.open(os.path.join(png_dir, image))
    if image.mode=='RGBA':
        image = image.convert('RGB')

    img_width, img_height = image.size
    target_size = [img_width, img_height]

    pred_bboxes = pred_bbox(bboxes)
    # image_data, legend_orientation = associate_legend_preview(pred_bboxes)

    # if isHbar:
    #     columns = [ylabel] + legend_names + ["xlabel", "ylabel", "title"]
    # else:
    #     columns = [xlabel] + legend_names + ["xlabel", "ylabel", "title"]

    # df = pd.DataFrame(columns=columns)

    # if isHbar:
    #     df[ylabel] = row_indexes
    # else:
    #     df[xlabel] = row_indexes

    # df["title"] = [title]*len(df)
    # df["xlabel"] = [xlabel]*len(df)
    # df["ylabel"] = [ylabel]*len(df)

    # df.to_csv(os.path.join(csv_dir, str(image_index) + ".csv"), index=False)


(2016-09) OECD 주요국가의 문화재정 비교_정보람_5.png True
